# How to exploit the month-end flow effect
Fund managers report their holdings every month. They don’t want to tell investors that they lost money the latest meme stock. So they will sell the meme stocks and buy higher quality assets, like bonds.

We might be able to take advantage of this effect by buying bonds toward the end of the month and selling them at the beginning.

In [ ]:
# import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import vectorbt as vbt

In [ ]:
# Load the data
tlt = vbt.YFData.download(
    "TLT", 
    start="2004-01-01"
).get("Close").to_frame()
close = tlt.Close